In [ ]:
import pandas as pd
import re
from pathlib import Path
import logging
import sys
from datetime import datetime

# -----------------------------
# Helper: make safe filenames
# -----------------------------
def safe_filename(name: str) -> str:
    name = name.strip()
    name = name.replace(" ", "_")
    name = re.sub(r'[\\/*?:"<>|]', "", name)
    return name

# -----------------------------
# Main logic
# -----------------------------
def main():
    input_dir = Path(r"D:\Balaji-workbench\exceltomd\input")   # 🔁 Folder with many Excel files

    # Create unique output folder per run
    run_id = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_root = Path(f"output_{run_id}")

    # ---- Create output root ----
    try:
        output_root.mkdir(exist_ok=True)
    except Exception:
        print("❌ Failed to create output directory.")
        sys.exit(1)

    # -----------------------------
    # Setup logging (per run folder)
    # -----------------------------
    log_file = output_root / "debug.log"

    logger = logging.getLogger()
    logger.setLevel(logging.DEBUG)

    # Clear existing handlers (important if re-running in same session)
    logger.handlers.clear()

    # File handler (DEBUG level)
    file_handler = logging.FileHandler(log_file, mode="w", encoding="utf-8")
    file_formatter = logging.Formatter("%(asctime)s [%(levelname)s] %(message)s")
    file_handler.setFormatter(file_formatter)
    file_handler.setLevel(logging.DEBUG)

    # Console handler (INFO level)
    console = logging.StreamHandler()
    console.setLevel(logging.INFO)
    console_formatter = logging.Formatter("[%(levelname)s] %(message)s")
    console.setFormatter(console_formatter)

    logger.addHandler(file_handler)
    logger.addHandler(console)

    logging.info(f"Logging to: {log_file}")
    logging.info("Starting Excel folder to CSV/MD conversion")

    # ---- Validation: check input dir exists ----
    if not input_dir.exists() or not input_dir.is_dir():
        logging.error(f"Input folder not found: {input_dir}")
        print(f"❌ Input folder not found: {input_dir}")
        sys.exit(1)

    excel_files = list(input_dir.glob("*.xlsx"))

    if not excel_files:
        logging.warning("No Excel files found in input folder.")
        print("⚠️ No Excel files found in input folder.")
        return

    logging.info(f"Found {len(excel_files)} Excel files")

    # ---- Process each Excel file ----
    for excel_file in excel_files:
        logging.info(f"Processing file: {excel_file.name}")

        file_stem = safe_filename(excel_file.name)  # keep .xlsx in folder name
        file_output_dir = output_root / file_stem

        try:
            file_output_dir.mkdir(exist_ok=True)
        except Exception:
            logging.exception(f"Failed to create folder for {excel_file.name}")
            print(f"❌ Failed to create folder for {excel_file.name}")
            continue

        # ---- Read Excel file ----
        try:
            sheets = pd.read_excel(excel_file, sheet_name=None)
        except Exception:
            logging.exception(f"Failed to read Excel file: {excel_file}")
            print(f"❌ Failed to read Excel file: {excel_file.name}")
            continue

        if not sheets:
            logging.warning(f"No sheets in file: {excel_file.name}")
            print(f"⚠️ No sheets in file: {excel_file.name}")
            continue

        # ---- Process each sheet ----
        for sheet_name, df in sheets.items():
            safe_sheet = safe_filename(sheet_name)

            base_name = safe_filename(excel_file.stem)
            csv_path = file_output_dir / f"{base_name}_{safe_sheet}.csv"
            md_path = file_output_dir / f"{base_name}_{safe_sheet}.md"

            # Save CSV
            try:
                df.to_csv(csv_path, index=False)
                logging.info(f"Saved CSV: {csv_path}")
            except Exception:
                logging.exception(f"Failed to save CSV: {csv_path}")
                print(f"❌ Failed to save CSV for {excel_file.name} - {sheet_name}")
                continue

            # Save Markdown
            try:
                with md_path.open("w", encoding="utf-8") as f:
                    f.write(f"# File: {excel_file.name}\n")
                    f.write(f"# Sheet: {sheet_name}\n\n")
                    f.write(df.to_markdown(index=False))
                logging.info(f"Saved MD: {md_path}")
            except Exception:
                logging.exception(f"Failed to save MD: {md_path}")
                print(f"❌ Failed to save MD for {excel_file.name} - {sheet_name}")
                continue

            print(f"✅ Exported: {csv_path} and {md_path}")

    logging.info("All Excel files processed successfully.")
    print("🎉 Done! All Excel files processed.")
    print(f"📝 Check log file: {log_file}")

# -----------------------------
# Entry point
# -----------------------------
if __name__ == "__main__":
    try:
        main()
    except Exception:
        logging.exception("Unexpected fatal error")
        print("❌ Unexpected error occurred. Check debug.log for details.")


[INFO] Logging to: output_20260211_150405\debug.log
[INFO] Starting Excel folder to CSV/MD conversion
[INFO] Found 1 Excel files
[INFO] Processing file: timesheet.xlsx
[INFO] Saved CSV: output_20260211_150405\timesheet.xlsx\timesheet_weekly_sheet.csv
[INFO] Saved MD: output_20260211_150405\timesheet.xlsx\timesheet_weekly_sheet.md
[INFO] Saved CSV: output_20260211_150405\timesheet.xlsx\timesheet_HL-BL.csv
[INFO] Saved MD: output_20260211_150405\timesheet.xlsx\timesheet_HL-BL.md
[INFO] Saved CSV: output_20260211_150405\timesheet.xlsx\timesheet_TimeBox_sheet.csv
[INFO] Saved MD: output_20260211_150405\timesheet.xlsx\timesheet_TimeBox_sheet.md
[INFO] All Excel files processed successfully.


✅ Exported: output_20260211_150405\timesheet.xlsx\timesheet_weekly_sheet.csv and output_20260211_150405\timesheet.xlsx\timesheet_weekly_sheet.md
✅ Exported: output_20260211_150405\timesheet.xlsx\timesheet_HL-BL.csv and output_20260211_150405\timesheet.xlsx\timesheet_HL-BL.md
✅ Exported: output_20260211_150405\timesheet.xlsx\timesheet_TimeBox_sheet.csv and output_20260211_150405\timesheet.xlsx\timesheet_TimeBox_sheet.md
🎉 Done! All Excel files processed.
📝 Check log file: output_20260211_150405\debug.log
